Tournament keys -- had to lookup online @ https://coverage-matrix.sportradar.com/

Wimbledon 
- MS: sr:competition:2555
- WS: sr:competition:2559


French
- MS: sr:competition:2579
- WS: sr:competition:2583


Australian
- MS: sr:competition:2567
- WS: sr:competition:2571


US
- MS: sr:competition:2591	
- WS: sr:competition:2595	

In [1]:
# Import packages
import http.client
import json
import pandas as pd
from datetime import datetime
import numpy as np
import time
import pickle

In [2]:
# Api keys
eda_api_key = "thp5rpgrbegb8hkc7zrapy36"
mds_api_key = "fqe4f9fyp8m84mg679xu7dnj"

# Variable to count api calls
global CALLS
CALLS = 0

# Make an API call given get, return a json object
def send_req(get, func):
    global CALLS
    # Connect to SR API
    conn = http.client.HTTPSConnection("api.sportradar.com")
    conn.request("GET", get)
    with conn.getresponse() as res:
        # Check if the response status is OK (200)
        if res.status == 200:
            data = res.read().decode("utf-8")
            
            # Check if data is not empty before decoding
            if data:
                CALLS += 1
                return json.loads(data)
            else:
                print("Error: Empty response data: " + func)
                return None
        else:
            print(f"Error: Failed request with status code {res.status}: {res.read().decode()}:" + func)
            return None

# Display json object neatly
def pp(json_obj):
    print(json.dumps(json_obj, indent=4))

# Grandslam competition codes for API
M_GRAND_SLAMS = {"sr:competition:2555":"2023","sr:competition:2579":"2023","sr:competition:2567":"2024", "sr:competition:2591":"2023"} # grandslams and their most current year
W_GRAND_SLAMS = {"sr:competition:2559":"2023","sr:competition:2583":"2023","sr:competition:2571":"2024","sr:competition:2595":"2023"}

# temp competition variable (Wimbledon)
#competition = M_GRAND_SLAMS.keys()[0]

In [3]:
# Get json object of the seasons (returns the top 3) related to the competition (tourney)
def get_competition_seasons(comp):
    func = "get_competition_seasons"
    json = send_req("/tennis/trial/v3/en/competitions/"+comp+"/seasons.json?api_key="+eda_api_key, func)
    if json != None:
        return json['seasons']
    else: return None

In [4]:
# Get the season ids and add to a list
# note that 2024 seasons have not happened yet so they're empty
# Theyre ordered chronologically
def return_seasons(seasons_json, comp_dict):
    # Create a dictionary of season ids and their years
    seasons = {} 
    if seasons_json==None:
        return seasons
    for sj in seasons_json:
        recent_year = comp_dict[sj['competition_id']]
        if sj['year']==recent_year: # only the most recent season has the pbp and enhanced stats
            seasons[sj['id']] = sj['name']
    return seasons

In [5]:
# Gets information about every match in the season (tournament)
def get_summary(sea, start=0):
    func = "get_summary"
    get = "/tennis/trial/v3/en/seasons/"+sea+"/summaries.json?api_key="+eda_api_key+"&start="+str(start)
    return send_req(get, func)

In [6]:
# Seeing which matches (after the qualifiers - commented out) have play by play stats
def list_of_pbps(summary):
    pbps = []
    if summary == None:
        return pbps
    for match in summary["summaries"]:
        #round = match['sport_event']['sport_event_context']['round']['name']
        # if round in ['round_of_64','round_of_32','round_of_16','semifinal','final',]: 
        props = match["sport_event"]['coverage']['sport_event_properties']
        if props['enhanced_stats'] and props['play_by_play']:
            pbps.append(match["sport_event"]["id"])
    return pbps

In [7]:

def matches_from_comp(competitions):
    # Given a competition, return a list of matches that have play by play
    matches = []

    for comp in competitions:
        time.sleep(2)
        seasons_json = get_competition_seasons(comp)
        time.sleep(2)
        seasons = return_seasons(seasons_json, competitions)
        for sea in list(seasons.keys()): # this isn't actually a list but it's fine
            print(sea)
            start = 0
            pbps = []
            # makes sure we get all of the matches in a tournament, query to multiple summary pages if possible
            while True:
                time.sleep(2)
                summary = get_summary(sea, start)
                if not summary or not summary['summaries']:  # break loop if empty
                    break
                pbps += list_of_pbps(summary)
                start += 200
            print("appending "+str(len(pbps))+" to matches")
            matches.append(pbps)

    print(CALLS)
    return matches
    

In [ ]:
# matches_m = matches_from_comp(M_GRAND_SLAMS)
# for x in matches_m:
#     print(len(x))

# gs_matches_men = sum(matches_m,[])

# with open('data/gs_matches_men.pkl', 'wb') as f:
#     pickle.dump(gs_matches_men, f)

127
127
127
127


In [8]:
matches_w = matches_from_comp(W_GRAND_SLAMS)
for x in matches_w:
    print(len(x))

gs_matches_women = sum(matches_w,[])

with open('data/gs_matches_women.pkl', 'wb') as f:
    pickle.dump(gs_matches_women, f)

sr:season:95309
appending 127 to matches
sr:season:94021
appending 127 to matches
sr:season:102229
appending 127 to matches
sr:season:97747
appending 127 to matches
16
127
127
127
127
